In [1]:
import pymongo
from datetime import datetime
from bson.objectid import ObjectId

#### Estabelecendo conexão cliente-servidor

In [2]:
# Criando conexão.
con = pymongo.MongoClient("localhost", 27017)

In [3]:
# Acessando o banco de dados cadastrodb.
db = con.cadastrodb

In [4]:
# Listando todos os banco de dados da conexão.
for dbs in con.list_database_names():
    print(dbs)

admin
cadastrodb
config
local


In [5]:
# Criando coleção posts no banco de dados.
posts = db.posts

# Listando as coleções criadas.
db.list_collection_names()

['posts']

In [6]:
# Limpando a coleção (caso tenha algum documento armazenado).
posts.delete_many({})

#### Inserindo documentos

In [7]:
# Criando três dicionários (documentos) que, sintaxecamente, assemelham-se às estruturas JSON.

post1 = {
    "codigo": "ID-9987725",
    "prod_name": "geladeira",
    "marcas": ["brastemp", "consul", "electrolux"],
    "data_cadastro": datetime(2022, 5, 20),
}

post2 = {
    "codigo": "ID-2209876",
    "prod_name": "televisor",
    "marcas": ["samsung", "panasonic", "lg"],
    "data_cadastro": datetime(2022, 5, 21),
}

post3 = {
    "codigo": "ID-2879521",
    "prod_name": "videogame",
    "marcas": ["playstation", "xbox", "nintendo"],
    "data_cadastro": datetime(2022, 5, 22),
}

many_posts = [post1, post2, post3]

In [8]:
# Inserindo os objetos na coleção posts.
posts_id = posts.insert_many(many_posts)

# Visualizando os ids de cada objeto.
posts_id.inserted_ids

[ObjectId('628c0d9aea72f1231e170822'),
 ObjectId('628c0d9aea72f1231e170823'),
 ObjectId('628c0d9aea72f1231e170824')]

In [9]:
# Contando a quantidade de documentos.
posts.count_documents({})

3

In [10]:
# Listando todos os documentos.
for item in posts.find():
    print(item, end="\n\n")

{'_id': ObjectId('628c0d9aea72f1231e170822'), 'codigo': 'ID-9987725', 'prod_name': 'geladeira', 'marcas': ['brastemp', 'consul', 'electrolux'], 'data_cadastro': datetime.datetime(2022, 5, 20, 0, 0)}

{'_id': ObjectId('628c0d9aea72f1231e170823'), 'codigo': 'ID-2209876', 'prod_name': 'televisor', 'marcas': ['samsung', 'panasonic', 'lg'], 'data_cadastro': datetime.datetime(2022, 5, 21, 0, 0)}

{'_id': ObjectId('628c0d9aea72f1231e170824'), 'codigo': 'ID-2879521', 'prod_name': 'videogame', 'marcas': ['playstation', 'xbox', 'nintendo'], 'data_cadastro': datetime.datetime(2022, 5, 22, 0, 0)}



#### Deletando documentos

In [11]:
# Deletando documentos com base em uma query.

query_delete = {"prod_name": "televisor"}

posts.delete_many(query_delete)

In [12]:
for item in posts.find():
    print(item, end="\n\n")

{'_id': ObjectId('628c0d9aea72f1231e170822'), 'codigo': 'ID-9987725', 'prod_name': 'geladeira', 'marcas': ['brastemp', 'consul', 'electrolux'], 'data_cadastro': datetime.datetime(2022, 5, 20, 0, 0)}

{'_id': ObjectId('628c0d9aea72f1231e170824'), 'codigo': 'ID-2879521', 'prod_name': 'videogame', 'marcas': ['playstation', 'xbox', 'nintendo'], 'data_cadastro': datetime.datetime(2022, 5, 22, 0, 0)}



#### Busca por _id

In [13]:
# _id no formato string.
str_id = str(posts.find_one()["_id"])

# _id no formato ObjectId.
obj_id = ObjectId(str_id)

In [14]:
# Não tem retorno.
posts.find_one({"_id": str_id})

In [15]:
# Retorna o documento.
posts.find_one({"_id": obj_id})

{'_id': ObjectId('628c0d9aea72f1231e170822'),
 'codigo': 'ID-9987725',
 'prod_name': 'geladeira',
 'marcas': ['brastemp', 'consul', 'electrolux'],
 'data_cadastro': datetime.datetime(2022, 5, 20, 0, 0)}

#### Busca com operador

In [16]:
date = datetime(2022, 5, 21)

# Busca pelo documentos com data de cadastro antes do dia 21/05/2022.
for item in posts.find({"data_cadastro": {"$lt": date}}):
    print(item, end="\n\n")

{'_id': ObjectId('628c0d9aea72f1231e170822'), 'codigo': 'ID-9987725', 'prod_name': 'geladeira', 'marcas': ['brastemp', 'consul', 'electrolux'], 'data_cadastro': datetime.datetime(2022, 5, 20, 0, 0)}



#### Utilizando índice para criar atributos únicos

In [17]:
# Criando índice único para o atributo codigo
db.posts.create_index([("codigo", pymongo.ASCENDING)], unique=True)

'codigo_1'

In [18]:
post_novo = {
    "codigo": "ID-2200690",
    "prod_name": "celular",
    "marcas": ["motorola", "apple", "xiaomi"],
    "data_cadastro": datetime.utcnow(),
}

post_duplicado = {
    "codigo": "ID-2879521",
    "prod_name": "videogame",
    "marcas": ["playstation", "xbox", "nintendo"],
    "data_cadastro": datetime(2022, 5, 22),
}

In [19]:
# Insere normalmente.
posts.insert_one(post_novo)

In [20]:
# Levanta excessão de chave duplicada.
try:
    posts.insert_one(post_duplicado)
except pymongo.errors.DuplicateKeyError:
    print("Erro de chave duplicada")

Erro de chave duplicada


In [21]:
for item in posts.find():
    print(item, end="\n\n")

{'_id': ObjectId('628c0d9aea72f1231e170822'), 'codigo': 'ID-9987725', 'prod_name': 'geladeira', 'marcas': ['brastemp', 'consul', 'electrolux'], 'data_cadastro': datetime.datetime(2022, 5, 20, 0, 0)}

{'_id': ObjectId('628c0d9aea72f1231e170824'), 'codigo': 'ID-2879521', 'prod_name': 'videogame', 'marcas': ['playstation', 'xbox', 'nintendo'], 'data_cadastro': datetime.datetime(2022, 5, 22, 0, 0)}

{'_id': ObjectId('628c0d9bea72f1231e170825'), 'codigo': 'ID-2200690', 'prod_name': 'celular', 'marcas': ['motorola', 'apple', 'xiaomi'], 'data_cadastro': datetime.datetime(2022, 5, 23, 22, 41, 31, 715000)}

